# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
##ONLY IF you have not installed plotnine before##
# !pip install plotnine (in anaconda prompt)
from plotnine import *

# Importing data

In [ ]:
eu = pd.read_csv('~/YOUR_PATH/NL_FR_BE_data_monthly.csv')
ff = pd.read_csv('~/YOUR_PATH/Europe_FF_Factors.csv')

# look at the data
display(eu)
display(ff)

# Defining sorting function

In [ ]:
# arguments are
# x: a vector of returns
# P: the number of portfolios we want
# we assign P=10 as a default value, so unless we want 
# a different number of portfolios than 10, 
# we do not need to set a value for P
def sortPort(x, P = 10):
    # assign a portfolio-number to each stock-observation in a particular month
    ranks = pd.qcut(x , P, labels = False, duplicates ='drop') + 1
    # return the portfolio-number and add a 'p' and a leading zero in front
    ranks = ranks.apply(lambda x: 'p'+str(int(x)).zfill(2) 
                        if not pd.isnull(x) else x)
    return ranks

# Calculating momentum returns

In [ ]:
# lag 11-month return by 2 months
eu['RET11_lag2'] = eu.groupby('ISIN')['RET11'].shift(2)
display(eu)

In [ ]:
# apply our sortPort function to each month (groupby 'mdate') separately
# here, we are using the lagged 11-month return as a sorting variable
eu['momport'] = eu.groupby(['mdate'])['RET11_lag2'].apply(lambda x: 
                                                          sortPort(x))
display(eu)

In [ ]:
# calculate (equal weighted) portfolio returns 
# for each of the 10 portfolios in each month
mom_portfolios = eu.groupby(['mdate','momport'])['RET'].mean()
mom_portfolios = mom_portfolios.reset_index()
display(mom_portfolios)

In [ ]:
# convert from long to wide format
mom_portfolios = mom_portfolios.pivot(index = 'mdate', 
                                      columns = 'momport', 
                                      values = 'RET')
display(mom_portfolios)

In [ ]:
# calculate long-short winner-loser portfolio (excess) return
mom_portfolios['MOM'] = mom_portfolios['p10'] - mom_portfolios['p01']
display(mom_portfolios)

# Analyze momentum returns

In [ ]:
# merge our portfolios with FF factors
portfolios = mom_portfolios.merge(ff, on = 'mdate')
display(portfolios)

In [ ]:
# run a Fama-French 3-factor regression
# notice: MOM is already an excess return (self-financing portfolio)
# if you want to run the regression on another portfolio, 
# e.g., the winners (p10)
# you need to sutract the risk-free rate! 
# => formula="p10-RF~MktRF+HML+SMB"
x = portfolios[['MktRF', 'HML', 'SMB']]
x = sm.add_constant(x)
reg = sm.OLS(portfolios['MOM'], x).fit(cov_type = 'HAC', 
                                       cov_kwds = {'maxlags': 11})

# summarize the regression result
display(reg.summary())

In [ ]:
# calculate the standard deviation of the residuals (for the information ratio)
residuals = reg.resid
sigma_epsilon = np.std(residuals)

# save regression coefficients
coefs = reg.params

In [ ]:
# Information Ratio (risk-adjusted abnormal returns)
print('Information Ratio =', np.round(coefs[0]/sigma_epsilon*np.sqrt(12),4))

# Sharpe Ratio (risk-adjusted returns)
print('Sharpe Ratio =', np.round(np.mean(portfolios['MOM'])/
                                 np.std(portfolios['MOM'])*np.sqrt(12),4))

# Plotting cumulated returns

In [ ]:
# convert portfolios returns to long format (plotnine ggplot requires this)
plotdata = portfolios.melt(id_vars = ['mdate'], 
                           var_name = 'portfolio', 
                           value_name = 'RET')
display(plotdata)

In [ ]:
# merge each return with risk-free rate in that month
plotdata = plotdata.merge(ff[['RF', 'mdate']], on = 'mdate')
display(plotdata)

In [ ]:
# calculate gross return = 1 + portfolio return (not excess!)
# so, since all of the factors are long-short portfolios,
# we need to add the risk-free rate 
# (that's what you earn on your collateral for the short-side)
# if we want to plot the 10 past-return-sorted portfolios later
# then we do not need to add the risk-free rate, 
# because these are not excess returns!
# so we filter by saying, whenever the portfolio is NOT in 
# this list of portfolio names, by using "in" and "not in"

ls_ports = ['MOM','MktRF','HML','SMB','WML']

def makeGrossReturn(plotdata):
    if plotdata['portfolio'] in ls_ports:
        return 1 + plotdata['RET'] + plotdata['RF']
    if plotdata['portfolio'] not in ls_ports:
        return 1 + plotdata['RET']

In [ ]:
plotdata['grossRET'] = plotdata.apply(makeGrossReturn, axis = 1)
display(plotdata)

In [ ]:
print(plotdata['mdate'].min())

In [ ]:
# set 1 dollar for first date, i.e., end of July 1991 (your initial investment)
initial_inv = plotdata.portfolio.unique()
initial_inv = pd.DataFrame(initial_inv, columns = ['portfolio'])
display(initial_inv)

In [ ]:
initial_inv[['mdate','return','RF','grossRET']] = [199107, None, None, 1]
display(initial_inv)

In [ ]:
plotdata1 = pd.concat([initial_inv, plotdata])
display(plotdata1.head(30))

In [ ]:
# calculate cumulated return (that's why we needed the 1+ earlier)
# now we get 1*(1+return_1stmonth)*(1+return_2ndmonth)*...
# so the cumulated amount of dollars of that portfolio
plotdata1['cumRET'] = plotdata1.groupby(['portfolio']).grossRET.cumprod()
display(plotdata1)

In [ ]:
# convert mdates to actual dates (plotnine ggplot then knows it is a date)
plotdata1['date'] = pd.to_datetime(
    plotdata1['mdate'],format='%Y%m') + pd.tseries.offsets.MonthEnd(1)
display(plotdata1)

In [ ]:
# define the portfolios to be plotted in 1st and 2nd plot
ports1 = ['MOM','SMB','HML','MktRF','RF']
ports2 = ['p01','p02','p03','p04','p05',
         'p06','p07','p08','p09','p10']

# Plots (Using plotnine)

In [ ]:
# create two ggplots: one that plots the cumulated returns 
# (of just MOM, MktRF, HML, SMB and RF) with a log(10)-scale
# and the second that plots p01 up to p10 also with a log(10)-scale
plot1 = plotdata1[plotdata1['portfolio'].isin(ports1)]
plot2 = plotdata1[plotdata1['portfolio'].isin(ports2)]

In [ ]:
print(
    ggplot(plot1)
    + aes(x = 'date', y = 'cumRET', color = 'portfolio')
    + geom_line()
    + xlab('Date')
    + ylab('Portfolio value in USD')
    + scale_y_log10()
    + scale_x_date(breaks = ('1 year'))
    + theme(axis_text_x = element_text(rotation = 90))
)

In [ ]:
print(
    ggplot(plot2)
    + aes(x = 'date', y = 'cumRET', color = 'portfolio')
    + geom_line()
    + xlab('Date')
    + ylab('Portfolio value in USD')
    + scale_y_log10()
    + scale_x_date(breaks = ('1 year'))
    + theme(axis_text_x = element_text(rotation = 90))
)

# Plots (Using matplotlib)

In [ ]:
# Convert portfolio values (plotdata1) to wide format (for matplotlib)
pltdata = plotdata1.pivot_table(index=['date'], 
                                columns='portfolio', 
                                values='cumRET').reset_index()

# create two ggplots: one that plots the cumulated returns 
# (of just MOM, MktRF, HML, SMB and RF) with a log(10)-scale
# and the second that plots p01 up to p10 also with a log(10)-scale

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Value of Investment')

for portfolio in pltdata:
    if portfolio in ports1:
        plt.plot(pltdata.date, pltdata[portfolio], label = portfolio)

plt.xlabel('Date')
plt.ylabel('Portfolio value in USD')
plt.legend()

plt.yscale('log')

plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Value of Investment')

for portfolio in pltdata:
    if portfolio != 'date':
        if portfolio in ports2:
            plt.plot(pltdata.date, 
                     pltdata[portfolio], 
                     label = portfolio)

plt.xlabel('Date')
plt.ylabel('Portfolio value in USD')
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5))

plt.yscale('log')

plt.show()